In [12]:
# Net class: Contains the name of the net and the faults that are associated with it
class Net:
    all = []  # List of all instances
    def __init__(self,name,type = None ,level = None,Gate = None,Gate_inputs = None):
        self.name = int(name)
        self.type = type
        self.faults = ["sa0","sa1"]  # Default faults: Fault Population
        self.level = level
        self.Gate = Gate
        self.Gate_inputs = Gate_inputs
        Net.all.append(self)         # Add instance to list of all instances
        
    def __repr__(self):
        return f"Net_{self.name}" 
    
    def disp_attributes(self):
        # To display all attributes of the instance
        print("-----------------------------------------")
        print(f"Net_{self.name} :")
        print(f"name = {self.name}")
        print(f"type = {self.type}")
        print(f"faults = {self.faults}")
        print(f"level = {self.level}")
        print(f"Gate = {self.Gate}")
        print(f"Gate_inputs = {self.Gate_inputs}")
        print("-----------------------------------------")
    
    @classmethod
    def get_instance(cls,name):
        for instance in cls.all:
            if instance.name == name:
                return instance
        return None
    
    @classmethod
    def check_instance(cls,name):
        for instance in cls.all:
            if instance.name == name:
                return True
        return False



In [13]:
with open("netlist.txt") as f:
    netlist = f.readlines()

# Cleaning the netlist
for i in range(len(netlist)):
    netlist[i] = netlist[i].split()
    netlist[i] = [x for x in netlist[i] if x != '']

print(netlist)

[['INPUT', '1', '2', '3', '4', '5'], ['OUTPUT', '11'], ['FANOUT', '7', '8', '9'], ['NAND', '6', '1', '2'], ['AND', '7', '4', '5'], ['AND', '10', '6', '3', '8'], ['OR', '11', '10', '9']]


In [14]:
# Levelise the netlist

# Make list of output nodes , input nodes for given netlist provided 

# find Primary inputs :

output_list = []
input_list = []
primary_inputs = []
primary_output = []



for item in netlist:
    if item[0] == "INPUT":
        for node in item[1:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                n.type = "PrimaryInput"
            else :
                Net(node,"PrimaryInput",0)
            primary_inputs.append(node)
            

    elif item[0] == "OUTPUT":
        if Net.check_instance(int(item[1])):
            n = Net.get_instance(int(item[1]))
            n.type = "PrimaryOutput"
        else:
            Net(item[1],"PrimaryOutput")

        primary_output.append(item[1])
        

    elif item[0] == "FANOUT":
        for node in item[2:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                n.type = "FanoutWire"
            else:
                Net(node,"FanoutWire")



    elif item[0] in ["AND","OR","NAND","NOR","NOT"]:

        output_list.append(item[1])

        # Handling output node 
        if Net.check_instance(int(item[1])):
            n = Net.get_instance(int(item[1]))
            if n.type == "PrimaryOutput":
                pass
        else:
            Net(item[1],"wire")

        Net.get_instance(int(item[1])).Gate_inputs = item[2:]

        # Handling input nodes
        for node in item[2:]:
            if Net.check_instance(int(node)):
                n = Net.get_instance(int(node))
                if n.type in ["FanoutWire","PrimaryInput"]:
                    pass
            else:
                Net(node,"wire",None,item[0])

            
            input_list.append(node)
    
    print(Net.all)

input_list = [ node for node in input_list if node not in primary_inputs]

print(output_list)
print(input_list)
print(primary_output)
print(primary_inputs)
print(Net.all)



[Net_1, Net_2, Net_3, Net_4, Net_5]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6, Net_7]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6, Net_7, Net_10]
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6, Net_7, Net_10]
['6', '7', '10', '11']
['6', '8', '10', '9']
['11']
['1', '2', '3', '4', '5']
[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6, Net_7, Net_10]


In [15]:
for obj in Net.all:
    obj.disp_attributes()
    

-----------------------------------------
Net_1 :
name = 1
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_2 :
name = 2
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_3 :
name = 3
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_4 :
name = 4
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_5 :
name = 5
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_11 :
name = 11
type = Pr

In [16]:
# Levelise the netlist

count = int(max(primary_inputs))+1
curr_level = 0

while count <= int(primary_output[0]):
    if str(count) not in output_list:
        obj = Net.get_instance(count)
        obj.level = curr_level
        count += 1
    else:
        obj = Net.get_instance(count)
        inp = obj.Gate_inputs
        inp = [int(x) for x in inp]
        inp_objects = [Net.get_instance(x) for x in inp]
        inp_obj_levels = [x.level for x in inp_objects]
        
        obj.level = max(inp_obj_levels)+1
        curr_level = obj.level
        count += 1



for obj in Net.all:
    obj.disp_attributes()      
    

-----------------------------------------
Net_1 :
name = 1
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_2 :
name = 2
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_3 :
name = 3
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_4 :
name = 4
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_5 :
name = 5
type = PrimaryInput
faults = ['sa0', 'sa1']
level = 0
Gate = None
Gate_inputs = None
-----------------------------------------
-----------------------------------------
Net_11 :
name = 11
type = Pr

In [17]:
print(Net.all)

x = Net.get_instance(1)

print(Net.check_instance(1))
print(Net.check_instance(5))

[Net_1, Net_2, Net_3, Net_4, Net_5, Net_11, Net_8, Net_9, Net_6, Net_7, Net_10]
True
True
